In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [83]:
def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Downloads/BED_files/srinivas_sir_covered.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/sen_spe_files_07_09_2023/truthset.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/truthset.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

In [85]:
truth = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/truthset.vcf',comment= '#', sep = '\t', header=None, low_memory=False)
truth.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
truth

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,856883,.,A,G,50,PASS,"platforms=3;platformnames=PacBio,Illumina,10X;...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:597:1,205:0,70:197"
1,chr1,930939,.,G,A,50,PASS,"platforms=4;platformnames=Illumina,PacBio,CG,1...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:634:0,214:39,328:234"
2,chr1,931131,.,C,CCCCT,50,PASS,"platforms=4;platformnames=CG,Illumina,PacBio,1...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:458:18,193:0,0:217"
3,chr1,941119,.,A,G,50,PASS,"platforms=4;platformnames=Illumina,PacBio,CG,1...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:559:0,208:0,252:263"
4,chr1,942335,.,C,G,50,PASS,"platforms=3;platformnames=Illumina,PacBio,10X;...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:637:0,241:0,283:197"
...,...,...,...,...,...,...,...,...,...,...
49823,chr22,50698821,.,T,C,50,PASS,"platforms=5;platformnames=Illumina,PacBio,CG,1...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:755:0,273:52,379:306"
49824,chr22,50739662,.,G,A,50,PASS,"platforms=6;platformnames=Illumina,PacBio,CG,1...",GT:PS:DP:ADALL:AD:GQ,"0/1:.:948:139,133:214,202:950"
49825,chr22,50744057,.,A,G,50,PASS,"platforms=4;platformnames=PacBio,Illumina,10X,...",GT:PS:DP:ADALL:AD:GQ,"1/1:.:613:67,243:0,73:278"
49826,chr22,50776600,.,C,G,50,PASS,"platforms=4;platformnames=PacBio,Illumina,10X,...",GT:PS:DP:ADALL:AD:GQ,"0/1:.:775:164,197:30,44:553"


In [7]:
import pandas as pd
import os

file_directory = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705/'

# Define a function to read and process a VCF file
def read_vcf_file(file_path):
    # Construct the full file path
    full_file_path = os.path.join(file_directory, file_path)

    # Read the VCF file into a Pandas DataFrame
    vcf_df = pd.read_csv(full_file_path, sep='\t', header=None, comment='#', low_memory=False, encoding='latin-1',
                        names=['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE'])
    return vcf_df[['CHROM', 'POS', 'REF', 'ALT']]

# List of VCF file names (without full paths)
vcf_files = ['12652705_BCFTOOL.vcf', '12652705_VARSCAN2.vcf', '12652705_MUTECT2.vcf', '12652705_HAPLOTYPECALLER.vcf', '12652705_DEEPVARIANT.vcf']

# Initialize the result DataFrame with the variants from the first VCF file
result_df = read_vcf_file(vcf_files[0])

# Iterate through the rest of the VCF files and merge with the result
for file_name in vcf_files[1:]:
    vcf_df = read_vcf_file(file_name)
    result_df = result_df.merge(vcf_df, on=['CHROM', 'POS', 'REF', 'ALT'], how='inner')

# Specify the full file path for the output Excel file
output_excel_path = os.path.join(file_directory, 'intersected_variants.xlsx')

# Save the result to an Excel file
result_df.to_excel(output_excel_path, index=False)

In [8]:
truth = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/output.tsv', sep = '\t')
truth = truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
truth_1 = truth.copy()
truth_1['ALT'] = truth_1['ALT'].str.split(',')
truth_1 = truth_1.explode('ALT')
truth_1

,CHROM,POS,REF,ALT
0,chr1,65490,G,A
1,chr1,65529,C,T
2,chr1,65532,A,T
3,chr1,65552,G,A
4,chr1,65556,C,T
...,...,...,...,...
23761809,chrY,57194235,T,C
23761810,chrY,57194236,G,C
23761811,chrY,57194239,A,C
23761812,chrY,57194242,G,T


In [9]:
intersection = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705/intersected_variants.xlsx')
intersection

,CHROM,POS,REF,ALT
0,chr1,930939,G,A
1,chr1,941119,A,G
2,chr1,944858,A,G
3,chr1,948245,A,G
4,chr1,952180,A,C
...,...,...,...,...
38586,chrX,154678060,G,C
38587,chrX,154766321,G,T
38588,chrX,154776873,G,A
38589,chrX,154781346,G,C


In [10]:
intersection.ALT.value_counts()

G    9862
C    9695
A    9549
T    9485
Name: ALT, dtype: int64

# BCFTOOL

In [54]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705_BCFTOOL.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf['DP'] = vcf['INFO'].str.extract(r'DP=(\d+)')[0].fillna('0').astype(int)
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[6]
vcf['RD'] = vcf['AD'].str.split(',').str[0].astype(int)
vcf['A_D'] = vcf['AD'].str.split(',').str[1].astype(int)
vcf['VAF'] = vcf['A_D'] / (vcf['RD'] + vcf['A_D'])
#vcf_vaf = vcf[vcf['VAF'] >= 0.01]
#vcf_after = vcf_vaf[vcf_vaf['DP'] >= 22]
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['gnomADe_SAS_AF'] = vcf['CSQ'].str.split('|').str[56]
vcf['gnomADe_AF'] = vcf['CSQ'].str.split('|').str[48]
vcf['gnomADe_AF'] = vcf['gnomADe_AF'].replace('', 0).astype('float')
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].replace('', 0)
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].astype('float')
vcf = vcf[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'VAF', 'gnomADe_AF', 'gnomADe_SAS_AF']]
vcf

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,826893,G,A,97,1.000000,0.818800,0.817500
1,chr1,930939,G,A,67,1.000000,0.000000,0.000000
2,chr1,941119,A,G,23,1.000000,0.933500,0.917300
3,chr1,942451,T,C,2,1.000000,0.999900,0.999700
4,chr1,944858,A,G,14,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
43793,chrX,154781346,G,C,44,0.555556,0.000565,0.005461
43794,chrX,154929926,T,G,86,0.462500,0.147000,0.337700
43795,chrY,1387425,C,G,65,1.000000,0.000000,0.000000
43796,chrY,1418109,C,G,111,1.000000,0.000000,0.000000


In [58]:
bcf_inter = pd.merge(intersection, vcf, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'left', sort = False)
bcf_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,67,1.000000,0.000000,0.000000
1,chr1,941119,A,G,23,1.000000,0.933500,0.917300
2,chr1,944858,A,G,14,1.000000,0.000000,0.000000
3,chr1,948245,A,G,73,1.000000,1.000000,1.000000
4,chr1,952180,A,C,36,1.000000,0.927700,0.918700
...,...,...,...,...,...,...,...,...
38586,chrX,154678060,G,C,32,0.541667,0.000000,0.000000
38587,chrX,154766321,G,T,150,1.000000,0.997900,1.000000
38588,chrX,154776873,G,A,71,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,44,0.555556,0.000565,0.005461


In [56]:
bcf_inter = bcf_inter[bcf_inter['VAF'] >= 0.1]
bcf_inter = bcf_inter[bcf_inter['DP'] >= 22]
bcf_inter = bcf_inter[bcf_inter['gnomADe_AF'] <= 0.5]
bcf_inter = bcf_inter[bcf_inter['gnomADe_AF'] <= 0.5]
bcf_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,67,1.000000,0.000000,0.000000
10,chr1,962933,C,T,56,0.538462,0.003222,0.026450
12,chr1,965125,G,C,210,1.000000,0.241100,0.301600
13,chr1,973443,G,A,33,1.000000,0.000000,0.000000
17,chr1,976536,C,T,44,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
38585,chrX,154546029,T,G,133,0.398305,0.004260,0.039690
38586,chrX,154678060,G,C,32,0.541667,0.000000,0.000000
38588,chrX,154776873,G,A,71,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,44,0.555556,0.000565,0.005461


In [57]:
bcf_truth = pd.merge(bcf_inter, truth_1, on=['CHROM', 'POS', 'REF', 'ALT'])
bcf_truth = bcf_truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
bcf_truth

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,67,1.000000,0.000000,0.000000
1,chr1,962933,C,T,56,0.538462,0.003222,0.026450
2,chr1,965125,G,C,210,1.000000,0.241100,0.301600
3,chr1,973443,G,A,33,1.000000,0.000000,0.000000
4,chr1,976536,C,T,44,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
23339,chrX,154546029,T,G,133,0.398305,0.004260,0.039690
23340,chrX,154678060,G,C,32,0.541667,0.000000,0.000000
23341,chrX,154776873,G,A,71,1.000000,0.146600,0.353400
23342,chrX,154781346,G,C,44,0.555556,0.000565,0.005461


# VARSCAN2

In [59]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705_VARSCAN2.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf['DP'] = vcf['SAMPLE'].str.split(':').str[3].fillna('0').astype(int)
vcf['RD'] = vcf['SAMPLE'].str.split(':').str[4].fillna('0').astype(int)
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[5].fillna('0').astype(int)
vcf['VAF'] = vcf['AD'] / (vcf['RD'] + vcf['AD'])
#vcf_vaf = vcf[vcf['VAF'] >= 0.01]
#vcf_after = vcf_vaf[vcf_vaf['DP'] >= 22]
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['gnomADe_SAS_AF'] = vcf['CSQ'].str.split('|').str[56]
vcf['gnomADe_AF'] = vcf['CSQ'].str.split('|').str[48]
vcf['gnomADe_AF'] = vcf['gnomADe_AF'].replace('', 0).astype('float')
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].replace('', 0)
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].astype('float')
vcf = vcf[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'VAF', 'gnomADe_AF', 'gnomADe_SAS_AF']]
vcf

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,69511,A,G,211,1.000000,0.9497,0.9854
1,chr1,826893,G,A,91,1.000000,0.8188,0.8175
2,chr1,930939,G,A,59,1.000000,0.0000,0.0000
3,chr1,941119,A,G,17,1.000000,0.9335,0.9173
4,chr1,944858,A,G,12,1.000000,0.0000,0.0000
...,...,...,...,...,...,...,...,...
42893,chrY,11986608,T,C,48,1.000000,0.0000,0.0000
42894,chrY,11986732,C,T,16,1.000000,0.0000,0.0000
42895,chrY,56961138,A,G,81,1.000000,0.0000,0.0000
42896,chrY,56961295,G,T,63,1.000000,0.0000,0.0000


In [60]:
var_inter = pd.merge(intersection, vcf, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'left', sort = False)
var_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,59,1.000000,0.000000,0.000000
1,chr1,941119,A,G,17,1.000000,0.933500,0.917300
2,chr1,944858,A,G,12,1.000000,0.000000,0.000000
3,chr1,948245,A,G,61,1.000000,1.000000,1.000000
4,chr1,952180,A,C,32,1.000000,0.927700,0.918700
...,...,...,...,...,...,...,...,...
38586,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
38587,chrX,154766321,G,T,133,1.000000,0.997900,1.000000
38588,chrX,154776873,G,A,63,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.542857,0.000565,0.005461


In [61]:
var_inter = var_inter[var_inter['VAF'] >= 0.1]
var_inter = var_inter[var_inter['DP'] >= 22]
var_inter = var_inter[var_inter['gnomADe_AF'] <= 0.5]
var_inter = var_inter[var_inter['gnomADe_AF'] <= 0.5]
var_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,59,1.000000,0.000000,0.000000
10,chr1,962933,C,T,52,0.538462,0.003222,0.026450
12,chr1,965125,G,C,201,1.000000,0.241100,0.301600
13,chr1,973443,G,A,30,1.000000,0.000000,0.000000
17,chr1,976536,C,T,39,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
38585,chrX,154546029,T,G,118,0.398305,0.004260,0.039690
38586,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
38588,chrX,154776873,G,A,63,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.542857,0.000565,0.005461


In [62]:
var_truth = pd.merge(var_inter, truth_1, on=['CHROM', 'POS', 'REF', 'ALT'])
var_truth = var_truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
var_truth

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,59,1.000000,0.000000,0.000000
1,chr1,962933,C,T,52,0.538462,0.003222,0.026450
2,chr1,965125,G,C,201,1.000000,0.241100,0.301600
3,chr1,973443,G,A,30,1.000000,0.000000,0.000000
4,chr1,976536,C,T,39,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
23006,chrX,154546029,T,G,118,0.398305,0.004260,0.039690
23007,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
23008,chrX,154776873,G,A,63,1.000000,0.146600,0.353400
23009,chrX,154781346,G,C,35,0.542857,0.000565,0.005461


# MUTECT2

In [75]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705_MUTECT2.vcf', comment= '#', sep = '\t', header=None, low_memory=False, encoding='latin-1')
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf['DP'] = vcf['SAMPLE'].str.split(':').str[3].fillna('0').astype(int)
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[1]
vcf['RD'] = vcf['AD'].str.split(',').str[0].astype(int)
vcf['A_D'] = vcf['AD'].str.split(',').str[1].astype(int)
vcf['VAF'] = vcf['A_D'] / (vcf['RD'] + vcf['A_D'])
#vcf_vaf = vcf[vcf['VAF'] >= 0.01]
#vcf_after = vcf_vaf[vcf_vaf['DP'] >= 22]
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['gnomADe_SAS_AF'] = vcf['CSQ'].str.split('|').str[56]
vcf['gnomADe_AF'] = vcf['CSQ'].str.split('|').str[48]
vcf['gnomADe_AF'] = vcf['gnomADe_AF'].replace('', 0).astype('float')
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].replace('', 0)
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].astype('float')
vcf = vcf[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'VAF', 'gnomADe_AF', 'gnomADe_SAS_AF']]
vcf

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,941119,A,G,23,1.000000,0.933500,0.917300
2,chr1,942451,T,C,1,1.000000,0.999900,0.999700
3,chr1,944858,A,G,14,1.000000,0.000000,0.000000
4,chr1,948245,A,G,63,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...
45455,chrX,154678117,G,A,36,0.277778,0.000000,0.000000
45456,chrX,154766321,G,T,141,1.000000,0.997900,1.000000
45457,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
45458,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [76]:
mut_inter = pd.merge(intersection, vcf, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'left', sort = False)
mut_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,941119,A,G,23,1.000000,0.933500,0.917300
2,chr1,944858,A,G,14,1.000000,0.000000,0.000000
3,chr1,948245,A,G,63,1.000000,1.000000,1.000000
4,chr1,952180,A,C,30,1.000000,0.927700,0.918700
...,...,...,...,...,...,...,...,...
38586,chrX,154678060,G,C,25,0.520000,0.000000,0.000000
38587,chrX,154766321,G,T,141,1.000000,0.997900,1.000000
38588,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [77]:
mut_inter = mut_inter[mut_inter['VAF'] >= 0.1]
mut_inter = mut_inter[mut_inter['DP'] >= 22]
mut_inter = mut_inter[mut_inter['gnomADe_AF'] <= 0.5]
mut_inter = mut_inter[mut_inter['gnomADe_AF'] <= 0.5]
mut_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
10,chr1,962933,C,T,52,0.519231,0.003222,0.026450
12,chr1,965125,G,C,189,1.000000,0.241100,0.301600
13,chr1,973443,G,A,35,1.000000,0.000000,0.000000
17,chr1,976536,C,T,40,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
38585,chrX,154546029,T,G,128,0.421875,0.004260,0.039690
38586,chrX,154678060,G,C,25,0.520000,0.000000,0.000000
38588,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [78]:
mut_truth = pd.merge(mut_inter, truth_1, on=['CHROM', 'POS', 'REF', 'ALT'])
mut_truth = mut_truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
mut_truth

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,962933,C,T,52,0.519231,0.003222,0.026450
2,chr1,965125,G,C,189,1.000000,0.241100,0.301600
3,chr1,973443,G,A,35,1.000000,0.000000,0.000000
4,chr1,976536,C,T,40,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
22970,chrX,154546029,T,G,128,0.421875,0.004260,0.039690
22971,chrX,154678060,G,C,25,0.520000,0.000000,0.000000
22972,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
22973,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


# DEEPVARIANT

In [79]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705_DEEPVARIANT.vcf', comment= '#', sep = '\t', header=None, low_memory=False, encoding='latin-1')
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf['DP'] = vcf['SAMPLE'].str.split(':').str[2].fillna('0').astype(int)
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[3]
vcf['RD'] = vcf['AD'].str.split(',').str[0].astype(int)
vcf['A_D'] = vcf['AD'].str.split(',').str[1].astype(int)
vcf['VAF'] = vcf['A_D'] / (vcf['RD'] + vcf['A_D'])
#vcf_vaf = vcf[vcf['VAF'] >= 0.01]
#vcf_after = vcf_vaf[vcf_vaf['DP'] >= 22]
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['gnomADe_SAS_AF'] = vcf['CSQ'].str.split('|').str[56]
vcf['gnomADe_AF'] = vcf['CSQ'].str.split('|').str[48]
vcf['gnomADe_AF'] = vcf['gnomADe_AF'].replace('', 0).astype('float')
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].replace('', 0)
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].astype('float')
vcf = vcf[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'VAF', 'gnomADe_AF', 'gnomADe_SAS_AF']]
vcf

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,69511,A,G,222,1.000,0.9497,0.9854
1,chr1,826893,G,A,90,1.000,0.8188,0.8175
2,chr1,856883,A,G,2,1.000,0.0000,0.0000
3,chr1,930939,G,A,66,1.000,0.0000,0.0000
4,chr1,941119,A,G,24,1.000,0.9335,0.9173
...,...,...,...,...,...,...,...,...
49235,chrY,11154005,C,T,2,1.000,0.0000,0.0000
49236,chrY,11986362,A,C,63,1.000,0.0000,0.0000
49237,chrY,11986608,T,C,52,1.000,0.0000,0.0000
49238,chrY,11986732,C,T,13,1.000,0.0000,0.0000


In [80]:
deep_inter = pd.merge(intersection, vcf, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'left', sort = False)
deep_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,66,1.000000,0.000000,0.000000
1,chr1,941119,A,G,24,1.000000,0.933500,0.917300
2,chr1,944858,A,G,14,1.000000,0.000000,0.000000
3,chr1,948245,A,G,74,1.000000,1.000000,1.000000
4,chr1,952180,A,C,36,1.000000,0.927700,0.918700
...,...,...,...,...,...,...,...,...
38586,chrX,154678060,G,C,26,0.538462,0.000000,0.000000
38587,chrX,154766321,G,T,150,1.000000,0.997900,1.000000
38588,chrX,154776873,G,A,74,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,42,0.547619,0.000565,0.005461


In [81]:
deep_inter = deep_inter[deep_inter['VAF'] >= 0.1]
deep_inter = deep_inter[deep_inter['DP'] >= 22]
deep_inter = deep_inter[deep_inter['gnomADe_AF'] <= 0.5]
deep_inter = deep_inter[deep_inter['gnomADe_AF'] <= 0.5]
deep_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,66,1.000000,0.000000,0.000000
10,chr1,962933,C,T,56,0.553571,0.003222,0.026450
12,chr1,965125,G,C,217,1.000000,0.241100,0.301600
13,chr1,973443,G,A,35,1.000000,0.000000,0.000000
17,chr1,976536,C,T,43,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
38585,chrX,154546029,T,G,137,0.408759,0.004260,0.039690
38586,chrX,154678060,G,C,26,0.538462,0.000000,0.000000
38588,chrX,154776873,G,A,74,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,42,0.547619,0.000565,0.005461


In [82]:
deep_truth = pd.merge(deep_inter, truth_1, on=['CHROM', 'POS', 'REF', 'ALT'])
deep_truth = deep_truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
deep_truth

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,66,1.000000,0.000000,0.000000
1,chr1,962933,C,T,56,0.553571,0.003222,0.026450
2,chr1,965125,G,C,217,1.000000,0.241100,0.301600
3,chr1,973443,G,A,35,1.000000,0.000000,0.000000
4,chr1,976536,C,T,43,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
23327,chrX,154546029,T,G,137,0.408759,0.004260,0.039690
23328,chrX,154678060,G,C,26,0.538462,0.000000,0.000000
23329,chrX,154776873,G,A,74,1.000000,0.146600,0.353400
23330,chrX,154781346,G,C,42,0.547619,0.000565,0.005461


# HAPLOTYPECALLER

In [71]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/VCC_OLD/12652705_HAPLOTYPECALLER.vcf', comment= '#', sep = '\t', header=None, low_memory=False, encoding='latin-1')
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf['DP'] = vcf['SAMPLE'].str.split(':').str[2].fillna('0').astype(int)
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[1]
vcf['RD'] = vcf['AD'].str.split(',').str[0].astype(int)
vcf['A_D'] = vcf['AD'].str.split(',').str[1].astype(int)
vcf['VAF'] = vcf['A_D'] / (vcf['RD'] + vcf['A_D'])
#vcf_vaf = vcf[vcf['VAF'] >= 0.01]
#vcf_after = vcf_vaf[vcf_vaf['DP'] >= 22]
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['gnomADe_SAS_AF'] = vcf['CSQ'].str.split('|').str[56]
vcf['gnomADe_AF'] = vcf['CSQ'].str.split('|').str[48]
vcf['gnomADe_AF'] = vcf['gnomADe_AF'].replace('', 0).astype('float')
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].replace('', 0)
vcf['gnomADe_SAS_AF'] = vcf['gnomADe_SAS_AF'].astype('float')
vcf = vcf[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'VAF', 'gnomADe_AF', 'gnomADe_SAS_AF']]
vcf

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,941119,A,G,23,1.000000,0.933500,0.917300
2,chr1,944858,A,G,14,1.000000,0.000000,0.000000
3,chr1,948245,A,G,63,1.000000,1.000000,1.000000
4,chr1,948711,C,G,15,0.266667,0.000000,0.000000
...,...,...,...,...,...,...,...,...
43895,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
43896,chrX,154766321,G,T,141,1.000000,0.997900,1.000000
43897,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
43898,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [72]:
haplo_inter = pd.merge(intersection, vcf, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'left', sort = False)
haplo_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,941119,A,G,23,1.000000,0.933500,0.917300
2,chr1,944858,A,G,14,1.000000,0.000000,0.000000
3,chr1,948245,A,G,63,1.000000,1.000000,1.000000
4,chr1,952180,A,C,30,1.000000,0.927700,0.918700
...,...,...,...,...,...,...,...,...
38586,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
38587,chrX,154766321,G,T,141,1.000000,0.997900,1.000000
38588,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [73]:
haplo_inter = haplo_inter[haplo_inter['VAF'] >= 0.1]
haplo_inter = haplo_inter[haplo_inter['DP'] >= 22]
haplo_inter = haplo_inter[haplo_inter['gnomADe_AF'] <= 0.5]
haplo_inter = haplo_inter[haplo_inter['gnomADe_AF'] <= 0.5]
haplo_inter

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
10,chr1,962933,C,T,52,0.519231,0.003222,0.026450
12,chr1,965125,G,C,190,1.000000,0.241100,0.301600
13,chr1,973443,G,A,35,1.000000,0.000000,0.000000
17,chr1,976536,C,T,40,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
38585,chrX,154546029,T,G,128,0.414062,0.004260,0.039690
38586,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
38588,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
38589,chrX,154781346,G,C,35,0.600000,0.000565,0.005461


In [74]:
haplo_truth = pd.merge(haplo_inter, truth_1, on=['CHROM', 'POS', 'REF', 'ALT'])
haplo_truth = haplo_truth.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT'])
haplo_truth

,CHROM,POS,REF,ALT,DP,VAF,gnomADe_AF,gnomADe_SAS_AF
0,chr1,930939,G,A,57,1.000000,0.000000,0.000000
1,chr1,962933,C,T,52,0.519231,0.003222,0.026450
2,chr1,965125,G,C,190,1.000000,0.241100,0.301600
3,chr1,973443,G,A,35,1.000000,0.000000,0.000000
4,chr1,976536,C,T,40,1.000000,0.158000,0.207800
...,...,...,...,...,...,...,...,...
22948,chrX,154546029,T,G,128,0.414062,0.004260,0.039690
22949,chrX,154678060,G,C,24,0.541667,0.000000,0.000000
22950,chrX,154776873,G,A,66,1.000000,0.146600,0.353400
22951,chrX,154781346,G,C,35,0.600000,0.000565,0.005461
